In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train

,latitud,longitud,Barrio,Ciudad,Area_total,Area_constr,MONEDA,Area_constr_cat,NroBanios,Dormitorios,Antiguedad,Cocheras,geohash_grado5,geohash_grado6,geohash_grado7,geohash_grado8,geohash_grado9,Precio_m2_total
0,-12.070790,-77.019494,La Victoria,Lima,325.0,237.0,S/,5.0,3.0,6.0,37.0,2.0,6mc5r,6mc5r6,6mc5r6s,6mc5r6s4,6mc5r6s4e,4146.153846
1,-12.077229,-76.929563,La Molina,Lima,2535.0,402.0,USD,14.0,4.0,4.0,21.0,4.0,6mc73,6mc733,6mc733m,6mc733mt,6mc733mt5,3500.000000
2,-12.086676,-76.930033,La Molina,Lima,1500.0,700.0,S/,19.0,5.0,4.0,20.0,5.0,6mc71,6mc71r,6mc71rt,6mc71rtq,6mc71rtq0,2688.000000
3,-12.150773,-77.020962,Barranco,Lima,600.0,768.0,USD,19.0,9.0,8.0,63.0,2.0,6mc4z,6mc4zk,6mc4zk5,6mc4zk50,6mc4zk509,12250.000000
4,-12.105563,-77.042509,San Isidro,Lima,420.0,820.0,USD,19.0,5.0,5.0,21.0,4.0,6mc5n,6mc5nu,6mc5nu5,6mc5nu5r,6mc5nu5rg,29166.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,-12.104434,-76.964697,Santiago de Surco,Lima,445.0,280.0,S/,7.0,3.0,4.0,29.0,2.0,6mc70,6mc70s,6mc70sk,6mc70skn,6mc70skn8,5595.505618
4976,-12.229004,-76.927719,Villa el Salvador,Lima,140.0,140.0,S/,1.0,3.0,6.0,16.0,1.0,6mc63,6mc63m,6mc63my,6mc63myb,6mc63myby,8328.571429
4977,-12.087945,-76.968836,Santiago de Surco,Lima,300.0,330.0,S/,10.0,4.0,4.0,28.0,3.0,6mc70,6mc70x,6mc70x3,6mc70x3n,6mc70x3n8,8760.000000
4978,-12.077680,-76.929917,La Molina,Lima,600.0,363.0,USD,12.0,4.0,4.0,17.0,3.0,6mc73,6mc733,6mc733m,6mc733m6,6mc733m66,4958.333333


In [3]:
#Estas columnas se van a encodear con LabelEncoder.
#Para los Geohash utilizaremos otros metodo.

to_encoded_columns = train.select_dtypes(include='object').filter(regex='^(?!geohash)')

#Creamos los artifacts.
import pickle
from sklearn.preprocessing import LabelEncoder
label_encoder_dict = {}

for column in to_encoded_columns:
    label_encoder = LabelEncoder()
    label_encoder.fit(train[column])
    label_encoder_dict[column] = label_encoder    
    train[column] = label_encoder.transform(train[column])
    
    #Para encodear test también.
    test[column] = label_encoder.transform(test[column])
    
with open('../data/label_encoder_dict.pkl','wb') as handle:
    pickle.dump(label_encoder_dict, handle, protocol = pickle.HIGHEST_PROTOCOL)

#### Encodeo de Geohashes:

Para poder encodear los Geohashes tenemos que, como son variables de alta cardinalidad, tenemos que buscar un método para poder asignar geohashes cercanos a coordenadas nuevas que aparezcan cuando se haga la inferencia. 

In [4]:
#Por ahora no usaré geohashes. Veremos primero cómo va el performance con un modelo simple.
train.to_csv('../data/train_encoded.csv',index=False)
test.to_csv('../data/test_encoded.csv', index=False)

tree = BallTree(test_points, metric = 'haversine')

results = tree.query_radius(test_points, r=10, return_distance=True)


#From stackoverflow
from sklearn.neighbors import BallTree

ratio_tierra = 6371000
ratio_test = 1300000

test_points = [[-12.011108,-77.053447],[-12.003141, -77.06475]]
test_points_rad = np.array([[np.radians(x[0]), np.radians(x[1])] for x in test_points])

tree = BallTree(test_points_rad, metric='haversine')

ind, results = tree.query_radius(test_points_rad, r = ratio_test/ratio_tierra, return_distance=True)

print(ind)
print(results * ratio_tierra/1000)


def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Para calcular la distancia entre dos ubicaciones en la tierra.
    En kilometros.
    """
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 4)

haversine_distance(-12.011108,-77.053447,-12.003141, -77.06475)


from sklearn.neighbors import BallTree
import numpy as np

rng = np.random.RandomState(1)
X = rng.random_sample((10,2))

tree = BallTree(X, leaf_size=1, metric='haversine')
dist, ind = tree.query(X[:1], k=2)
print(ind)
print(dist)